# 카메라 좌표를 월드 좌표로 변환 예제
### ------------------------------------------------------------
### 좌표계/부호 규약 (중요!)
### - World(지도) 좌표: ENU (x=East, y=North, z=Up)
### - Camera 좌표: x=Right, y=Down, z=Forward  (CV 관례)
### - yaw: z(Up)축 기준 반시계(좌) + ; 0°일 때 동쪽(East) 향함
### - pitch: y(North)축 기준 + 이면 "기수 올림"(하늘쪽),  - 이면 "하향"
### - roll: x(East)축 기준 + 이면 오른쪽 날개 내림(우측으로 구름)
### (IMU/PX4가 NED을 쓰거나 부호규약이 다르면 알려줘! 변환 코드 붙여줄게.)
### ------------------------------------------------------------

### 1. 기본 import + 회전/좌표 변환 유틸

In [1]:
import numpy as np
import math


def rot_zyx(yaw_deg=0.0, pitch_deg=0.0, roll_deg=0.0):
    """
    R_wc = Rz(yaw) @ Ry(pitch) @ Rx(roll)
    - world <- camera  회전행렬 (카메라벡터를 월드좌표로 회전)
    - 각도 단위: degree
    """
    y, p, r = np.deg2rad([yaw_deg, pitch_deg, roll_deg])
    cz, sz = np.cos(y), np.sin(y)
    cy, sy = np.cos(p), np.sin(p)
    cx, sx = np.cos(r), np.sin(r)

    Rz = np.array([[cz, -sz, 0],
                   [sz,  cz, 0],
                   [ 0,   0, 1]], dtype=float)
    Ry = np.array([[ cy, 0, sy],
                   [  0, 1,  0],
                   [-sy, 0, cy]], dtype=float)
    Rx = np.array([[1,  0,   0],
                   [0, cx, -sx],
                   [0, sx,  cx]], dtype=float)

    return Rz @ Ry @ Rx

def get_R_bc(mount="nadir"):
    """
    Camera(OpenCV: +x=right, +y=down, +z=forward) -> Body(FLU: +X=forward, +Y=left, +Z=up)
    """
    # 전방(정면) 장착
    R_bc_front = np.array([
        [ 0,  0,  1],   # cZ -> bX
        [-1,  0,  0],   # cX -> -bY
        [ 0, -1,  0],   # cY -> -bZ
    ], float)

    # 하향(나디르) 장착
    R_bc_nadir = np.array([
        [ 0, -1,  0],
        [-1,  0,  0],
        [ 0,  0, -1],
    ], float)

    if mount == "front":
        return R_bc_front
    elif mount == "nadir":
        return R_bc_nadir
    else:
        raise ValueError("mount must be 'front' or 'nadir'")

def cam_to_world_point(X, Y, Z, yaw_deg, pitch_deg, roll_deg, Cw, mount="nadir", R_bc=None):
    """
    카메라 좌표 P_c=[X,Y,Z] (OpenCV: x=오른, y=아래, z=앞)를
    월드(ENU) 좌표로 변환: P_w = (R_wb @ R_bc) @ P_c + C_w
    - mount: 'nadir'(기본) 또는 'front'
    - R_bc: 직접 보정행렬을 넣고 싶으면 전달(우선순위 높음)
    """
    R_wb = rot_zyx(yaw_deg, pitch_deg, roll_deg)        # Body -> World
    R_bc = R_bc if R_bc is not None else get_R_bc(mount) # Camera -> Body
    R_wc = R_wb @ R_bc                                  # Camera -> World

    Pc = np.array([X, Y, Z], dtype=float)
    Cw = np.array(Cw, dtype=float)
    Pw = R_wc @ Pc + Cw
    return Pw

def ground_intersection_from_pixel(u, v, fx, fy, cx, cy,
                                   yaw_deg, pitch_deg, roll_deg,
                                   Cw, ground_z=0.0):
    """
    픽셀(u,v)에서 나가는 광선이 지면 z=ground_z와 만나는 점을 계산.
    (깊이값 없이도 '고도+자세'만으로 지면좌표 얻고 싶을 때 사용)
    반환: (Pg, rho), Pg=[x_e, y_n, z_g], rho=수평거리(ENU 평면에서의 거리)
    """
    # 1) 카메라 좌표에서 정규화 광선
    dx = (u - cx) / fx
    dy = (v - cy) / fy
    ray_cam = np.array([dx, dy, 1.0], dtype=float)
    ray_cam /= (np.linalg.norm(ray_cam) + 1e-12)

    # 2) 월드 회전
    R_wc = rot_zyx(yaw_deg, pitch_deg, roll_deg)
    ray_w = R_wc @ ray_cam

    # 3) Cw에서 z=ground_z 교점
    Cw = np.array(Cw, dtype=float)
    rwz = ray_w[2]
    if abs(rwz) < 1e-12:
        return None, None  # 광선이 거의 수평 -> 교점 불가(또는 매우 멀리)
    t = (ground_z - Cw[2]) / rwz
    if t <= 0:
        return None, None  # 광선이 위로 향하거나 카메라 뒤쪽
    Pg = Cw + t * ray_w
    rho = float(np.hypot(Pg[0]-Cw[0], Pg[1]-Cw[1]))
    return Pg, rho

### 2. 픽셀→카메라 3D 변환 유틸

In [2]:
def pixel_to_cam(u, v, Z, fx, fy, cx, cy):
    """
    깊이 Z(미터)가 있을 때 픽셀(u,v) → 카메라 좌표(X,Y,Z)
    카메라 좌표: x=Right, y=Down, z=Forward
    """
    X = (u - cx) / fx * Z
    Y = (v - cy) / fy * Z
    return float(X), float(Y), float(Z)

### 3. 예시 변수 세팅 (드론 포즈/카메라 내부/디텍션)
### ** 두 가지 시나리오 **

### (1) 이미 X,Y,Z(카메라 좌표)가 있는 경우
* (X=0.21, Y=0.05, Z=2.39가 나온 상황)

In [3]:
# --- 드론(카메라) 포즈: ENU 기준 ---
# 드론이 (E=100 m, N=200 m, U=50 m) 지점에서, 동쪽에서 북쪽으로 30° 틀어(동북 방향) 지면을 20° 내려다보며 수평(롤 0°)을 유지하고 있는 상태

Cw = [100.0, 200.0, 5.0]     # 드론의 월드 좌표 (E, N, U) [m]
yaw_deg = 0.0                # 방위(0°=East, +CCW)
pitch_deg = -10.0             # 하향이면 -, 상향이면 +
roll_deg = 0.0

# --- 카메라 좌표에서의 객체 위치 (이미 계산돼 있다고 가정) ---
X, Y, Z = 0.21, 0.05, 2.39    # meters


In [4]:
# 월드 좌표로 변환
# 도(degree)를 라디안(radian)으로 원소별 변환해요.
# 수학·프로그래밍에서 삼각함수와 회전 공식의 표준 단위가 라디안이기 때문에 삼각함수 자체는 항상 라디안이라 생각하면 안전
# 삼각함수(np.sin, np.cos 등)는 라디안 입력을 기대하므로, yaw/pitch/roll을 바로 쓰기 전에 보통 np.deg2rad로 바꿔줍니다.
y, p, r = np.deg2rad([yaw_deg, pitch_deg, roll_deg])
print(f"y: {y} / p: {p} / r: {r}")

y: 0.0 / p: -0.17453292519943295 / r: 0.0


In [5]:
cz, sz = np.cos(y), np.sin(y)
cy, sy = np.cos(p), np.sin(p)
cx, sx = np.cos(r), np.sin(r)

print(cz, sz)

1.0 0.0


### *** 코사인 사인 변환 값 의미? ***
  ![alt text](111-3.png)

In [6]:
Rz = np.array([[cz, -sz,  0],
               [sz,  cz,  0],
               [ 0,   0,  1]], dtype=float)

Ry = np.array([[ cy,   0, sy],
               [  0,   1,  0],
               [-sy,   0, cy]], dtype=float)

Rx = np.array([[ 1,   0,   0],
               [ 0,  cx, -sx],
               [ 0,  sx,  cx]], dtype=float)

R_wc = Rz @ Ry @ Rx

print(Rz)
print("*****" * 10)
print(Rz @ Ry @ Rx)

### print(Rz @ Ry @ Rx)는 z축→y축→x축 순서로 합성한 회전행렬을 출력하는 거예요.
# 즉, roll(Rx) → pitch(Ry) → yaw(Rz) 를(오른쪽부터 왼쪽으로) 차례로 적용한 최종 자세(orientation) 를 나타내는 3×3 행렬

[[ 1. -0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
**************************************************
[[ 0.98480775  0.         -0.17364818]
 [ 0.          1.          0.        ]
 [ 0.17364818  0.          0.98480775]]


### *** Rz 값의 의미는?? ***
  ![alt text](111-4.png)

### *** Rz @ Ry @ Rx 값의 의미?? ***
![alt text](111-5.png)

### *** 정면 장착과 하향 장착 테스트 ***
* "어떤 보정을 써야할까???" 
![alt text](333.png)

In [7]:
def test_d_w2(yaw_deg, pitch_deg, roll_deg, r_c):
    y, p, r = np.deg2rad([yaw_deg, pitch_deg, roll_deg])
    cz, sz = np.cos(y), np.sin(y)
    cy, sy = np.cos(p), np.sin(p)
    cx, sx = np.cos(r), np.sin(r)

    Rz = np.array([[cz,-sz,0],[sz,cz,0],[0,0,1]], float)
    Ry = np.array([[cy,0,sy],[0,1,0],[-sy,0,cy]], float)
    Rx = np.array([[1,0,0],[0,cx,-sx],[0,sx,cx]], float)
    R_wb = Rz @ Ry @ Rx

    R_bc_front = np.array([[0,0,1],[-1,0,0],[0,-1,0]], float)
    R_bc_nadir = np.array([[0,-1,0],[-1,0,0],[0,0,-1]], float)

    for name, R_bc in [("front", R_bc_front), ("nadir", R_bc_nadir)]:
        R_wc = R_wb @ R_bc
        d_w = R_wc @ (r_c / np.linalg.norm(r_c))
        print(name, "d_w[2] =", d_w[2])

# 예) 방금 얻은 카메라 벡터(사람 bbox 중심 3D)
r_c = np.array([X, Y, Z], float)  # 예: [0.21, 0.05, 2.39]
test_d_w2(yaw_deg, pitch_deg, roll_deg, r_c)

front d_w[2] = 0.15242501711597992
nadir d_w[2] = -0.984433154558511


In [8]:
### 상기 테스트 결과에 따라 보정 선택 ###
R_bc = np.array([
    [ 0, -1,  0],
    [-1,  0,  0],
    [ 0,  0, -1],
], dtype=float)

R_wc = R_wc @ R_bc
R_wc

array([[ 0.        , -0.98480775,  0.17364818],
       [-1.        ,  0.        ,  0.        ],
       [ 0.        , -0.17364818, -0.98480775]])

### *** R_bc 를 행렬곱 하는 이유는?? ***
![alt text](111-6.png)
![alt text](222-3.png)

In [9]:
# 카메라에서 얻은 방향벡터/점 (정규화해서 방향만 봐도 OK)
r_c = np.array([X, Y, Z], float)
d_w = R_wc @ (r_c / np.linalg.norm(r_c))
print("월드 U축 성분:", d_w[2])  # 지면을 보면 보통 d_w[2] < 0 (아래) 여야 함

월드 U축 성분: -0.984433154558511


In [10]:
##### 카메라 좌표의 점 P_c=[X,Y,Z]를 월드 좌표로 변환. #####
##### 드론 위치 Cw = [100.0, 200.0, 5.0] #####

Pw = R_wc @ np.array([X, Y, Z], dtype=float) + np.array(Cw, dtype=float)
Pw

array([100.36577876, 199.79      ,   2.63762706])

In [11]:
R  = float(np.linalg.norm([X, Y, Z]))  # 카메라-객체 직선거리
R

2.3997291513835473

### *** np.linalg.norm([X, Y, Z]) 의미?? ***
![alt text](111-7.png)

### (B) 픽셀(u,v) + 깊이 Z 로부터 X,Y,Z를 만들고 월드로
* (딥스/YOLO 바운딩박스 중심 등에서 얻은 픽셀 좌표와 Z가 있을 때)

In [12]:
def pixel_to_cam(u, v, Z, fx, fy, cx, cy):
    """
    깊이 Z(미터)가 있을 때 픽셀(u,v) → 카메라 좌표(X,Y,Z)
    카메라 좌표: x=Right, y=Down, z=Forward
    """
    X = (u - cx) / fx * Z
    Y = (v - cy) / fy * Z
    return float(X), float(Y), float(Z)

In [13]:
# --- 카메라 내부 파라미터 (예시: main_depth_infer_coord_extract.py 로그 값) ---
fx = 1016.79
fy = 1016.79
cx = 640.0
cy = 360.0

# --- 예시 픽셀/깊이 ---
u = 728.0
v = 382.0
Z = 2.39  # m (ROI 중앙값 등)

# 픽셀→카메라 3D
Xb, Yb, Zb = pixel_to_cam(u, v, Z, fx, fy, cx, cy)
print(Xb, Yb, Zb)

0.2068470382281494 0.05171175955703735 2.39


In [14]:
Pw_b = cam_to_world_point(Xb, Yb, Zb, yaw_deg, pitch_deg, roll_deg, Cw, mount="nadir", R_bc=None)
Pw_b

array([100.364093  , 199.79315296,   2.63732982])

In [15]:
def gravity_scale_correction(depth_m, fx, fy, cx, cy,
                             cam_height, yaw_deg, pitch_deg, roll_deg,
                             roi_bottom_frac=0.35, stride=4, pct_clip=(10,90)):
    """
    단일 프레임 depth를 '고도+자세'로 전역 스케일 보정.
    - 바닥은 영상 하단 roi_bottom_frac (디폴트 하단 35%)로 가정
    - stride로 샘플링해 계산량 감소
    """
    H, W = depth_m.shape
    y0 = int(H * (1.0 - roi_bottom_frac))
    ys = np.arange(y0, H, stride)
    xs = np.arange(0, W, stride)
    if len(ys)==0 or len(xs)==0:
        return depth_m, 1.0, {"used":0}

    # 카메라에서 본 '월드 업'의 방향(=지면 법선) n_c
    R_wc = rot_zyx(yaw_deg, pitch_deg, roll_deg)
    n_w  = np.array([0.0, 0.0, 1.0])
    n_c  = R_wc.T @ n_w   # plane normal in camera frame

    # 샘플 그리드
    uu, vv = np.meshgrid(xs, ys)
    Zp = depth_m[vv, uu]
    valid = np.isfinite(Zp) & (Zp > 0)

    dx = (uu - cx) / fx
    dy = (vv - cy) / fy
    denom = n_c[0]*dx + n_c[1]*dy + n_c[2]*1.0  # n·v, v=[dx,dy,1]
    # 광선이 지면을 향해야 교차(denom < 0) → 하향 카메라면 보통 음수
    valid &= (denom < -1e-6)

    if not np.any(valid):
        return depth_m, 1.0, {"used":0}

    Z_expected = -cam_height / denom[valid]       # 이론적 지면 교차 깊이(카메라 z)
    Z_pred     = Zp[valid]
    # 이상치 컷
    lo, hi = np.percentile(Z_expected/Z_pred, pct_clip)
    sc = (Z_expected/Z_pred)
    sc = sc[(sc>=lo) & (sc<=hi)]
    if sc.size == 0:
        return depth_m, 1.0, {"used":0}

    scale = float(np.median(sc))
    depth_corr = depth_m * scale
    return depth_corr, scale, {"used": int(sc.size), "lo": float(lo), "hi": float(hi)}


In [ ]:
# infer 끝난 뒤: depth_m (m), fx, fy, cx, cy 준비된 상태
cam_height = Cw[-1]      # 드론 AGL (m) - 실제 값 넣으세요

depth_m_corr, scale, info = gravity_scale_correction(
    depth_m, fx, fy, cx, cy,
    cam_height=cam_height,
    yaw_deg=yaw_deg, pitch_deg=pitch_deg, roll_deg=roll_deg,
    roi_bottom_frac=0.35, stride=4
)
print(f"[gravity-scale] scale={scale:.3f}, used={info['used']}")
print(f"[depth_m_corr] ={depth_m_corr}")

ValueError: not enough values to unpack (expected 2, got 1)